In [1]:
%load_ext dotenv
%dotenv

In [5]:
# TODO: test whirligig ws
# TODO: test with ccxt

import logging
import os
from zeta_py.zeta_client.accounts.pricing import Pricing
from solana.rpc.async_api import AsyncClient
from zeta_py.client import Client
from zeta_py.types import Network, Asset, Side
import anchorpy
from solders.pubkey import Pubkey
from zeta_py import pda
from solana.rpc.commitment import Confirmed, Finalized
from datetime import datetime, timedelta
from zeta_py.types import OrderOptions, OrderType
from solana.rpc.types import TxOpts
from datetime import datetime, timedelta
from zeta_py.types import OrderOptions, OrderType
from zeta_py import utils

logger = logging.getLogger("zeta_py")
logging.basicConfig(level=logging.DEBUG)

endpoint = os.getenv("ENDPOINT", "https://api.mainnet-beta.solana.com")
print(endpoint.split(".com")[0])
conn = AsyncClient(endpoint, commitment=Confirmed, blockhash_cache=False)

wallet = anchorpy.Wallet.local()  # get local filesystem keypair wallet
print(wallet.public_key)

https://zeta.rpcpool
4mYxEXiFwe3ePEmoj7CYCbiW7wGTojUQCmdPqGaoVaUh


In [3]:
# print(f"Balance: {(await conn.get_balance(wallet.public_key)).value / 1e9} SOL")

In [4]:
tx_opts = TxOpts({"preflight_commitment": Confirmed, "skip_confirmation": False})

client = await Client.load(Network.MAINNET, conn, wallet, [Asset.SOL], subscribe=True, tx_opts=tx_opts)

INFO:zeta_py.accounts.Account:Loaded account: State
INFO:zeta_py.accounts.Account:Loaded account: Pricing
INFO:zeta_py.market.Market.SOL:Subscribed to SOL:bid
INFO:zeta_py.market.Market.SOL:Subscribed to SOL:ask
INFO:zeta_py.accounts.Account:Loaded account: Clock
INFO:zeta_py.accounts.Account:Subscribed to Clock
INFO:zeta_py.accounts.Account:Subscribed to Pricing
INFO:zeta_py.accounts.Account:Loaded account: CrossMarginAccount


ERROR:zeta_py.accounts.Account:Error subscribing to Clock: 
ERROR:zeta_py.accounts.Account:Error subscribing to Pricing: 


In [5]:
print(client.balance)
print(client.positions)
print(client.open_orders)

0.1
{<Asset.SOL: 0>: Position(size=0.0, cost_of_trades=0.0)}
{<Asset.SOL: 0>: [Order(order_id=387381643994644657538064724, client_id=69, open_order_address=Pubkey(
    ciwBQfLdM97vFp8GwYYLczPgYoLfdkTVD6dhHAeGoWj,
), open_order_slot=0, fee_tier=0, info=OrderInfo(price=21.0, size=0.002, price_lots=21000000, size_lots=2), side=<Side.Bid: 0>, tif_offset=45301), Order(order_id=405828369621610135659486892, client_id=69, open_order_address=Pubkey(
    ciwBQfLdM97vFp8GwYYLczPgYoLfdkTVD6dhHAeGoWj,
), open_order_slot=1, fee_tier=0, info=OrderInfo(price=22.0, size=0.002, price_lots=22000000, size_lots=2), side=<Side.Ask: 1>, tif_offset=45301)]}


In [4]:
await client.deposit(0.1)

INFO:zeta_py.accounts.Account:Loaded account: CrossMarginAccountManager
INFO:zeta_py.client.Client:Depositing 0.1 USDC to margin account


Signature(
    4z7zNwWhqM6Fr4N2tuvccfSfJc3vBWTVZZgab1VV6qmFhZxPjHVeZ1tMyQjCX9CrLvFZsppuRzPRqysQVDxJ1Zr6,
)

In [ ]:
# Place order

expiry_ts = int((datetime.now() + timedelta(seconds=1000)).timestamp())
order_opts = OrderOptions(expiry_ts=expiry_ts, order_type=OrderType.PostOnlySlide, client_order_id=69)
await client.place_order(Asset.SOL, 0.1, 0.001, Side.Ask, order_opts)

# Quoting
Use this code to atomically cancel and replace a set of quotes on Zeta.

*For this example we quote 0.001 SOL-PERP contracts at 20bps spread*

In [7]:
# Quoting parameters
asset = Asset.SOL  # SOL-PERP market
quote_size = 0.001  # min size
quote_bps = 10  # basis points from midpoint
time_in_force = 10  # expire the quote after 10 seconds

# Get the latest mark price and calculate bid/ask prices
price_fixed = client.exchange.pricing.account.mark_prices[asset.to_index()]
bid_price = utils.convert_fixed_int_to_decimal(price_fixed * (1 - quote_bps / 10000))
ask_price = utils.convert_fixed_int_to_decimal(price_fixed * (1 + quote_bps / 10000))

# Set order options
expiry_ts = int((datetime.now() + timedelta(seconds=time_in_force)).timestamp())
order_opts = OrderOptions(expiry_ts=expiry_ts, order_type=OrderType.Limit, client_order_id=69)

# Execute quote!
await client.replace_quote(Asset.SOL, bid_price, quote_size, ask_price, quote_size, order_opts)

INFO:zeta_py.client.Client:Replacing SOL orders: 0.001x Bid @ $21.757750469999998, 0.001x Ask @ $21.801309529999997


Signature(
    3Lb5vqF3pQKSqXBLm4hx4EMgRtMJctcSuwgZ8unRfXF9wgFewQuXF7JMTpgDDURQMcPMfMLy1XifVD9P6o6KgMDd,
)

In [8]:
open_orders = await client.fetch_open_orders(Asset.SOL)
print("Open Orders:")
for order in open_orders:
    print(f"{order.side.name} {order.info.size}x ${order.info.price}")

Open Orders:
Bid 0.001x $21.7577
Ask 0.001x $21.8013


ERROR:zeta_py.accounts.Account:Error subscribing to Clock: no close frame received or sent
ERROR:zeta_py.accounts.Account:Error subscribing to Pricing: no close frame received or sent


In [ ]:
oid = open_orders[0].order_id
await client.cancel_order(Asset.SOL, order_id=oid, side=Side.Ask)

In [17]:
import ccxt.async_support as ccxt

binance = ccxt.binance()
symbol = "SOL/USDT"
info = await binance.fetch_ticker(symbol)
info["info"]["bidPrice"]

ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x110241650>


'21.86000000'